<a href="https://colab.research.google.com/github/changsin/Kaggle30Days/blob/main/12-14.intermediate_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intermediate Machine Learning
## Day 12 of Kaggle


## Missing values
Here is a handy way to check missing values.

In [ ]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

## Categorical variables

- To select only numerical columns

In [ ]:
 [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

- to apply OrdinalEncoder

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply ordinal encoder 
# Your code here
# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(label_X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(label_X_valid[good_label_cols])

# Check your answer
step_2.b.check()

### Submission

The submission required cleaning up the test data: i.e., imputing NaN values from certain columns.

In [ ]:
from sklearn.impute import SimpleImputer

final_imputer = SimpleImputer(strategy='most_frequent')
imputed_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
imputed_X_test = pd.DataFrame(final_imputer.transform(X_test))
imputed_X_test.columns = X_test.columns

In [ ]:
OH_cols_test = pd.DataFrame(OH_encoder.transform(imputed_X_test[low_cardinality_cols]))
OH_cols_test.index = imputed_X_test.index
num_X_test = imputed_X_test.drop(object_cols, axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

OH_X_test.head()

- Check to make sure that there are no missing values

In [ ]:
missing_val_count_by_column = (OH_X_test.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

- This step is not necessary since ordinal encoder is not the one we used, but leaving here just for illustration purposes.

In [ ]:
drop_X_test = X_test[[cname for cname in X_test.columns if X_test[cname].dtype in ['int64', 'float64']]]
label_X_test = imputed_X_test.drop(bad_label_cols, axis=1)
label_X_test[good_label_cols] = ordinal_encoder.transform(label_X_test[good_label_cols])

- Finally train, predict, and save for submission.

In [ ]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(OH_X_train, y_train)

# Get validation predictions and MAE
preds_test = model.predict(OH_X_test)

# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)